# Prediction of sales

### Problem Statement
[The dataset](https://drive.google.com/file/d/1B07fvYosBNdIwlZxSmxDfeAf9KaygX89/view?usp=sharing) represents sales data for 1559 products across 10 stores in different cities. Also, attributes of each product and store are available. The aim is to build a predictive model and determine the sales of each product at a particular store.

|Variable|Description|
|------------- |-------------|
|Item_Identifier|Unique product ID|
|Item_Weight|Weight of product|
|Item_Fat_Content|Whether the product is low fat or not|
|Item_Visibility|The % of total display area of all products in a store allocated to the particular product|
|Item_Type|The category to which the product belongs|
|Item_MRP|Maximum Retail Price (list price) of the product|
|Outlet_Identifier|Unique store ID|
|Outlet_Establishment_Year|The year in which store was established|
|Outlet_Size|The size of the store in terms of ground area covered|
|Outlet_Location_Type|The type of city in which the store is located|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|
|Item_Outlet_Sales|Sales of the product in the particulat store. This is the outcome variable to be predicted.|

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.



### In following sessions, we will explore the problem in following stages:

1. **Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome**
2. **Data Exploration – looking at categorical & continuous feature summaries and making inferences about the data**
3. **Data Cleaning – imputing missing values in the data and checking for outliers**
4. **Feature Engineering – modifying existing variables and/or creating new ones for analysis**
5. **Model Building – making predictive models on the data**
---------

In [58]:
import pandas as pd
import numpy as np

#Read files:
data = pd.read_csv("processed_data_p1.csv", delimiter=',')

## 4. Feature Engineering

1. Resolving the issues in the data to make it ready for the analysis.
2. Create some new variables using the existing ones.





### Create a broad category of Type of Item

`Item_Type` variable has many categories which might prove to be very useful in analysis. Look at the `Item_Identifier`, i.e. the unique ID of each item, it starts with either FD, DR or NC. If you see the categories, these look like being Food, Drinks and Non-Consumables. 

**Task:** Use the Item_Identifier variable to create a new column

In [59]:
products = data.groupby('Item_Identifier').size()
products

Item_Identifier
DRA12    6
DRA24    7
DRA59    8
DRB01    3
DRB13    5
        ..
NCZ30    7
NCZ41    5
NCZ42    5
NCZ53    5
NCZ54    7
Length: 1559, dtype: int64

In [60]:
data['Item_Category'] = data['Item_Identifier'].apply(lambda x: x[:2])
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,FD
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC


In [61]:
categ = data.groupby('Item_Category').size()
categ 

Item_Category
DR     799
FD    6125
NC    1599
dtype: int64

### Determine the years of operation of a store

**Task:** Make a new column depicting the years of operation of a store (i.e. how long the store exists). 

In [62]:
data['Years_Of_Operation'] = 2023 - data['Outlet_Establishment_Year'] #can be used Date time
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category,Years_Of_Operation
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD,24
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR,14
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD,24
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,FD,25
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC,36


### Modify categories of Item_Fat_Content

**Task:** There are difference in representation in categories of Item_Fat_Content variable. This should be corrected.

In [63]:
fat_cat = data.groupby('Item_Fat_Content').size()
fat_cat

Item_Fat_Content
LF          316
Low Fat    5089
Regular    2889
low fat     112
reg         117
dtype: int64

In [64]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'})


In [65]:
fat_cat = data.groupby('Item_Fat_Content').size()
fat_cat

Item_Fat_Content
Low Fat    5517
Regular    3006
dtype: int64

**Task:** There are some non-consumables as well and a fat-content should not be specified for them. Create a separate category for such kind of observations. 

NC 'Item_Category' 

In [66]:
data.loc[data['Item_Category'] == 'NC', 'Item_Fat_Content'] = 'Inedible'

In [67]:
fat_cat = data.groupby('Item_Fat_Content').size()
fat_cat

Item_Fat_Content
Inedible    1599
Low Fat     3918
Regular     3006
dtype: int64

### Numerical and One-Hot Encoding of Categorical variables

Since scikit-learn algorithms accept only numerical variables, we need to **convert all categorical variables into numeric types.** 

- if the variable is Ordinal we can simply map its values into numbers
- if the variable is Nominal (we cannot sort the values) we need to One-Hot Encode them --> create dummy variables

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  Item_Category              8523 non-null   object 
 13  Years_Of_Operation         8523 non-null   int64

In [69]:
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category,Years_Of_Operation
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD,24
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR,14
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD,24
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,FD,25
4,NCD19,8.930,Inedible,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834,FD,36
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,Medium,Tier 2,Supermarket Type1,549.2850,FD,21
8520,NCJ29,10.600,Inedible,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136,NC,19
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976,FD,14


In [70]:
data['Item_Fat_Content_Numbers'] = data['Item_Fat_Content'].replace({'Low Fat': 1, 'Regular':2, 'Inedible':3})

data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category,Years_Of_Operation,Item_Fat_Content_Numbers
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD,24,1
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR,14,2
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD,24,1
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,FD,25,2
4,NCD19,8.93,Inedible,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC,36,3


In [71]:
fat_cat = data.groupby('Item_Fat_Content_Numbers').size()
fat_cat

Item_Fat_Content_Numbers
1    3918
2    3006
3    1599
dtype: int64

In [72]:
cat_items = data.groupby('Item_Category').size()
cat_items

Item_Category
DR     799
FD    6125
NC    1599
dtype: int64

In [73]:
data['Item_Category_Numbers'] = data['Item_Category'].map({'DR': 1, 'FD':2, 'NC':3})


In [74]:
shops_types =  data.groupby('Outlet_Type').size()
shops_types 

Outlet_Type
Grocery Store        1083
Supermarket Type1    5577
Supermarket Type2     928
Supermarket Type3     935
dtype: int64

In [75]:
data['Outlet_Type_Num'] = data['Outlet_Type'].map({'Grocery Store': 0, 'Supermarket Type1':1, 'Supermarket Type2':2, 
                                                   'Supermarket Type3':3,})


In [76]:
shops_types =  data.groupby('Outlet_Type_Num').size()
shops_types 

Outlet_Type_Num
0    1083
1    5577
2     928
3     935
dtype: int64

In [77]:
item_type = data.groupby('Item_Type').size()
item_type

Item_Type
Baking Goods              648
Breads                    251
Breakfast                 110
Canned                    649
Dairy                     682
Frozen Foods              856
Fruits and Vegetables    1232
Hard Drinks               214
Health and Hygiene        520
Household                 910
Meat                      425
Others                    169
Seafood                    64
Snack Foods              1200
Soft Drinks               445
Starchy Foods             148
dtype: int64

In [78]:
data['Item_Type_Num'] = data['Item_Type'].map({'Baking Goods': 0, 'Breads':1, 'Breakfast':2, 
                                                   'Canned':3, 'Dairy':4, 'Frozen Foods':5, 'Fruits and Vegetables':6,
                                                    'Hard Drinks':7, 'Health and Hygiene':8, 'Household':9,
                                                      'Meat':10, 'Others':11, 'Seafood':12, 'Snack Foods':13, 'Soft Drinks':14,
                                                      'Starchy Foods':14 })



In [79]:
location = data.groupby('Outlet_Location_Type').size()
location

#Tier 1 cities such as New York or Los Angeles are highly developed, 
#Tier 2 cities such as Seattle or Pittsburgh are still developing their real estate markets, and 
# Tier 3 cities such as Akron or Biloxi have underdeveloped markets.

Outlet_Location_Type
Tier 1    2388
Tier 2    2785
Tier 3    3350
dtype: int64

In [80]:
data['Outlet_Location_Type_num'] = data['Outlet_Location_Type'].map({'Tier 1':1, 'Tier 2':2, 'Tier 3':3})


In [81]:
id_store = data.groupby('Outlet_Identifier').size()
id_store

Outlet_Identifier
OUT010    555
OUT013    932
OUT017    926
OUT018    928
OUT019    528
OUT027    935
OUT035    930
OUT045    929
OUT046    930
OUT049    930
dtype: int64

In [82]:
data['Outlet_ID'] = data['Outlet_Identifier'].map({'OUT010':0, 'OUT013':1, 'OUT017':2, 
                                                   'OUT018':3, 'OUT019':4, 'OUT027':5, 
                                                   'OUT035':6, 'OUT045':7,
                                                   'OUT046':8, 'OUT049':9})



In [83]:
size = data.groupby('Outlet_Size').size()
size

Outlet_Size
High       932
Medium    5203
Small     2388
dtype: int64

In [84]:
data['Outlet_Size_Num'] = data['Outlet_Size'].map({'High': 2, 'Medium':1, 'Small':0})

data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,...,Item_Outlet_Sales,Item_Category,Years_Of_Operation,Item_Fat_Content_Numbers,Item_Category_Numbers,Outlet_Type_Num,Item_Type_Num,Outlet_Location_Type_num,Outlet_ID,Outlet_Size_Num
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,...,3735.1380,FD,24,1,2,1,4,1,9,1
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,...,443.4228,DR,14,2,1,2,14,3,3,1
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,...,2097.2700,FD,24,1,2,1,10,1,9,1
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,...,732.3800,FD,25,2,2,0,6,3,0,1
4,NCD19,8.93,Inedible,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,...,994.7052,NC,36,3,3,1,9,3,1,2


In [85]:
#saving data before final preparation
data.to_csv("processed_data_fe_p2.csv", index=False)

Delite or reorganizing data columns 

In [86]:
list(data) 

['Item_Identifier',
 'Item_Weight',
 'Item_Fat_Content',
 'Item_Visibility',
 'Item_Type',
 'Item_MRP',
 'Outlet_Identifier',
 'Outlet_Establishment_Year',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type',
 'Item_Outlet_Sales',
 'Item_Category',
 'Years_Of_Operation',
 'Item_Fat_Content_Numbers',
 'Item_Category_Numbers',
 'Outlet_Type_Num',
 'Item_Type_Num',
 'Outlet_Location_Type_num',
 'Outlet_ID',
 'Outlet_Size_Num']

In [87]:
# Delete column from the DataFrame
columns_drop = ['Item_Identifier','Item_Fat_Content','Item_Type', 'Outlet_Identifier', 'Outlet_Size','Outlet_Type',
                           'Outlet_Location_Type', 'Item_Category']
processed_data = data.drop(columns_drop, axis=1)

processed_data.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Years_Of_Operation,Item_Fat_Content_Numbers,Item_Category_Numbers,Outlet_Type_Num,Item_Type_Num,Outlet_Location_Type_num,Outlet_ID,Outlet_Size_Num
0,9.30,0.016047,249.8092,1999,3735.1380,24,1,2,1,4,1,9,1
1,5.92,0.019278,48.2692,2009,443.4228,14,2,1,2,14,3,3,1
2,17.50,0.016760,141.6180,1999,2097.2700,24,1,2,1,10,1,9,1
3,19.20,0.000000,182.0950,1998,732.3800,25,2,2,0,6,3,0,1
4,8.93,0.000000,53.8614,1987,994.7052,36,3,3,1,9,3,1,2


In [88]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                8523 non-null   float64
 1   Item_Visibility            8523 non-null   float64
 2   Item_MRP                   8523 non-null   float64
 3   Outlet_Establishment_Year  8523 non-null   int64  
 4   Item_Outlet_Sales          8523 non-null   float64
 5   Years_Of_Operation         8523 non-null   int64  
 6   Item_Fat_Content_Numbers   8523 non-null   int64  
 7   Item_Category_Numbers      8523 non-null   int64  
 8   Outlet_Type_Num            8523 non-null   int64  
 9   Item_Type_Num              8523 non-null   int64  
 10  Outlet_Location_Type_num   8523 non-null   int64  
 11  Outlet_ID                  8523 non-null   int64  
 12  Outlet_Size_Num            8523 non-null   int64  
dtypes: float64(4), int64(9)
memory usage: 865.7 KB


**All variables should be by now numeric.**

---------
### Exporting Data

**Task:** You can save the processed data to your local machine as a csv file. **We will use this dataset in the following sessions to create machine learning models.**

In [89]:
processed_data.to_csv("processed_data_part2.csv", index=False)